In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import UnitaryGate
import numpy as np

# --- Create oracle gate from user-defined matrix ---
def create_oracle_gate(oracle_matrix):
    """
    Takes a 4x4 unitary matrix and wraps it as a Gate in Qiskit.
    """
    return UnitaryGate(oracle_matrix, label="Oracle")

# --- Prompt user for 4x4 matrix ---
def input_unitary_matrix():
    print("Enter the 16 comma-separated values for the 4x4 unitary matrix:")
    #matrix_values = input().strip().split(',')
    matrix_values = "1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1".split(',')
    oracle_matrix = np.array([
        [float(matrix_values[i]) for i in range(4)],
        [float(matrix_values[i]) for i in range(4, 8)],
        [float(matrix_values[i]) for i in range(8, 12)],
        [float(matrix_values[i]) for i in range(12, 16)]
    ])
    return oracle_matrix

# --- Deutsch's Algorithm ---
def deutsch_algorithm(oracle_gate):
    qc = QuantumCircuit(2, 1)

    # Initialize output qubit in |1>
    qc.x(1)

    # Hadamard on both qubits
    qc.h([0, 1])

    # Apply oracle
    qc.append(oracle_gate, [0, 1])

    # Hadamard on input qubit only
    qc.h(0)

    # Measure first qubit
    qc.measure(0, 0)

    return qc

# --- Run simulation ---
oracle_matrix = input_unitary_matrix()
oracle_gate = create_oracle_gate(oracle_matrix)
qc = deutsch_algorithm(oracle_gate)

simulator = Aer.get_backend("qasm_simulator")
compiled = transpile(qc, simulator)
result = simulator.run(compiled, shots=10).result()
counts = result.get_counts()

print("Measurement outcomes:", counts)

# Determine constant or balanced
function_type = "constant" if "0" in counts and counts["0"] == 10 else "balanced"
print("Function type:", function_type)

print("\nCircuit:")
print(qc.draw())


Enter the 16 comma-separated values for the 4x4 unitary matrix:
Measurement outcomes: {'0': 10}
Function type: constant

Circuit:
     ┌───┐     ┌─────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0        ├┤ H ├┤M├
     ├───┤┌───┐│  Oracle │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1        ├──────╫─
     └───┘└───┘└─────────┘      ║ 
c: 1/═══════════════════════════╩═
                                0 
